In [1]:
import scipy.io
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as ptch
import matplotlib.image as mpimg
from PIL import Image
import weightedstats as ws
import math
from scipy.stats import norm
from numpy.random import random
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.datasets import load_sample_image
from sklearn.utils import shuffle
import os
import cv2

def multinomial_resample(weights):

    weights=weights.T
    cumulative_sum = np.cumsum(weights)
    #cumulative_sum[-1] = 1.  # avoid round-off errors: ensures sum is exactly one
    #print ( np.searchsorted(cumulative_sum, random(len(weights))))
    return np.searchsorted(cumulative_sum, random(len(weights)))


def lecture_image(tt) :

    SEQUENCE = "./seq2/"
    #charge le nom des images de la séquence
    filenames = os.listdir(SEQUENCE)
    filenames.sort(key=lambda x: int(x.split('.')[0][2:]))
    T = len(filenames)

    #charge la premiere image dans ’im’

    im=Image.open((str(SEQUENCE)+str(filenames[tt])))
    plt.imshow(im)

    return(im,filenames,T,SEQUENCE)

def selectionner_zone() :

    #lecture_image()
    print('Cliquer 4 points dans l image pour definir la zone a suivre.') ;
    zone = np.zeros([2,4])
    #   print(zone))
    compteur=0
    while(compteur != 4):
        res = plt.ginput(1)
        a=res[0]
        #print(type(a)))
        zone[0,compteur] = a[0]
        zone[1,compteur] = a[1]
        plt.plot(a[0],a[1],marker='X',color='red')
        compteur = compteur+1

        #print(zone)
    newzone = np.zeros([2,4])
    newzone[0, :] = np.sort(zone[0, :])
    newzone[1, :] = np.sort(zone[1, :])

    zoneAT = np.zeros([4])
    zoneAT[0] = newzone[0,0]
    zoneAT[1] = newzone[1,0]
    zoneAT[2] = newzone[0,3]-newzone[0,0]
    zoneAT[3] = newzone[1,3]-newzone[1,0]
    #affichage du rectangle
    #print(zoneAT)
    xy=(zoneAT[0],zoneAT[1])
    rect=ptch.Rectangle(xy,zoneAT[2],zoneAT[3],linewidth=3,edgecolor='red',facecolor='None')
    #plt.Rectangle(zoneAT[0:1],zoneAT[2],zoneAT[3])
    currentAxis = plt.gca()
    currentAxis.add_patch(rect)
    plt.show(block=False)
    return(zoneAT)


def rgb2ind(im,nb) :
    #nb = nombre de couleurs ou kmeans qui contient la carte de couleur de l'image de référence
    print(im)
    image=np.array(im,dtype=np.float64)/255
    w,h,d=original_shape=tuple(image.shape)
    image_array=np.reshape(image,(w*h,d))
    image_array_sample=shuffle(image_array,random_state=0)[:1000]
    print(image_array_sample.shape)
    # print(type(image_array))
    if type(nb)==int :
        kmeans=KMeans(n_clusters=nb,random_state=0).fit(image_array_sample)
    else :
        kmeans=nb

    labels=kmeans.predict(image_array)
    #print(labels)
    image=recreate_image(kmeans.cluster_centers_,labels,w,h)
    #print(image)
    return(Image.fromarray(image.astype('uint8')),kmeans)

def recreate_image(codebook,labels,w,h):
    d=codebook.shape[1]
    #image=np.zeros((w,h,d))
    image=np.zeros((w,h))
    label_idx=0
    for i in range(w):
        for j in range(h):
            #image[i][j]=codebook[labels[label_idx]]*255
            image[i][j]=labels[label_idx]
            #print(image[i][j])
            label_idx+=1

    return image



def calcul_histogramme(im,zoneAT,Nb):

    #  print(zoneAT)
    box=(zoneAT[0],zoneAT[1],zoneAT[0]+zoneAT[2],zoneAT[1]+zoneAT[3])
    # print(box)
    littleim = im.crop(box)
    ##    plt.imshow(littleim)
    ##    plt.show()
    new_im,kmeans= rgb2ind(littleim,Nb)
    histogramme=np.asarray(new_im.histogram())
    ##  print(histogramme)
    histogramme=histogramme/np.sum(histogramme)
    #  print(new_im)
    return (new_im,kmeans,histogramme)
##
N=100
Nb=10
landa=50
C1=300
C2=300
sar=lecture_image(0)
selecz=selectionner_zone()
jsp0,jsp1,refmap = calcul_histogramme(sar[0],selecz,Nb)
print(selecz)
inip=np.random.multivariate_normal([selecz[0],selecz[1]],np.sqrt(300),(N,2))
def distance(map) :
    s=0
    for i in range(Nb) :
        s += np.sqrt(map[i]*refmap[i])
    return(1-s)

def weight(particles,k) :
    l=[]
    for i in range(N):
        l.append(np.exp(-landa*distance(calcul_histogramme(lecture_image(k)[0],[particles[i][0],particles[i][1],selecz[2],selecz[3]],Nb)[2])))
    return(l)

w=weight(inip,0)
w=np.array(w)
w=w/w.sum()
w=w.flatten()
X_l=[inip]
unique_colors, counts = np.unique(sar[0], return_counts=True, axis=2)
print("hihihi")
print(unique_colors.shape[2])
plt.clf()
plt.ion()
X=inip
meanlist=[]
medianlist=[]
maxlist=[]
wl=[w]
for i in range(1, 40):
    abs=np.average(X[:,0]) #,weights=w)
    ord=np.average(X[:,1]) #,weights=w)
    meanlist.append((abs,ord))
    abs1=X[np.argmax(w),0]
    ord1=X[np.argmax(w),0]
    maxlist.append((abs1,ord1))
    abs2=ws.weighted_median(X[:,0],weights=w)
    ord2=ws.weighted_median(X[:,1],weights=w)
    medianlist.append((abs2,ord2))
    meanlist.append(distance(calcul_histogramme(lecture_image(i)[0],[abs,ord,selecz[2],selecz[3]],Nb)[2]))
    maxlist.append(distance(calcul_histogramme(lecture_image(i)[0],[abs1,ord1,selecz[2],selecz[3]],Nb)[2]))
    medianlist.append(distance(calcul_histogramme(lecture_image(i)[0],[abs2,ord2,selecz[2],selecz[3]],Nb)[2]))
    for j in range(N) :
        if w[j] > 0.001 :
            rect = ptch.Rectangle((X[j, 0], X[j, 1]), selecz[2], selecz[3], linewidth=3, edgecolor='red', facecolor='None')
            plt.imshow(lecture_image(i - 1)[0])
            currentAxis = plt.gca()
            currentAxis.add_patch(rect)
        # Add text label with the weight of the particle
            plt.text(X[j, 0], X[j, 1], f'{w[j]:.3f}', ha='center', va='center', color='white')
    rect2=ptch.Rectangle((abs2,ord2),selecz[2], selecz[3], linewidth=3, edgecolor='green', facecolor='None')
    rect1 = ptch.Rectangle((abs1, ord1), selecz[2], selecz[3], linewidth=3, edgecolor='blue', facecolor='None')
    rect = ptch.Rectangle((abs, ord), selecz[2], selecz[3], linewidth=3, edgecolor='red', facecolor='None')
    print(lecture_image(i - 1)[0])
    plt.imshow(lecture_image(i - 1)[0])
    currentAxis = plt.gca()
    currentAxis.add_patch(rect)
    currentAxis.add_patch(rect2)
    currentAxis.add_patch(rect1)
    #Pause for a brief moment to allow the figure to update
    plt.savefig('./croisement/frame%d.jpg' % i)
    #plt.show()
    plt.clf()  # Clear the figure
    A = np.random.choice(range(N), N, p=w)
    reech = X[A]
    U = np.random.normal(0,50,(N,2))
    X = reech + U
    print(X)
    w = weight(X, i)
    X_l.append(X)
    w = np.array(w)
    w = w / w.sum()
    w = w.flatten()
    wl.append(w)





Cliquer 4 points dans l image pour definir la zone a suivre.
<PIL.Image.Image image mode=RGB size=39x43 at 0x7FE3B0B179A0>
(1000, 3)
[412.38311688 106.92857143  38.44155844  42.5974026 ]


ValueError: cov must be 2 dimensional and square

In [ ]:
#plt.plot(meanlist, color="green")
#plt.plot(medianlist, color="brown")
#plt.plot(maxlist, color="blue")
neff=[]
for i in range(40) :
    s=0
    for j in range(50) :
        s += wl[i][j]**2
    neff.append(1/s)
plt.clf()
plt.plot(neff)
plt.show()

In [ ]:
print(maxlist)
print(medianlist)